# Environment Setup

In [1]:
# Install HappyML
import os

if not os.path.isdir("HappyML"):
    os.system("git clone https://github.com/cnchi/HappyML.git")

In [2]:
# Upload Dataset
Dataset_File = "Meals.csv"

if not os.path.isfile(Dataset_File):
  os.system("wget https://raw.githubusercontent.com/cnchi/datasets/master/" + Dataset_File)

In [3]:
# Customizable Constants
# Hyperparameter
train_size = 0.75

# hyperparameters of RNN
win_size = 2
sample_step = 1
win_moving = 1

data_batch = 10

# Data Preprocessing

In [4]:
# Load Data
import HappyML.preprocessor as pp
import numpy as np

dataset = pp.dataset(file="Meals.csv")

# Remove the date, keep meal only
dataset = pp.onehot_encoder(dataset, columns=[1]).iloc[:, 1:]

In [5]:
# Keep the One Hot Encoding Mapping
# Map number to real name of meals
mapper = [s[5:] for s in list(dataset.columns)]

# Convert dataset from DataFrame as NDArray
dataset = dataset.values

# The order of data matters in RNN, so we split data in order
train_data, test_data = np.split(dataset, [int(train_size * len(dataset))])

# Preparing RNN Dataset

In [9]:
# Training Set & Testing Set for RNN
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

train_set = TimeseriesGenerator(
  data=train_data,
  targets=train_data,
  length=win_size,
  sampling_rate=sample_step,
  stride=win_moving,
  batch_size=data_batch)

test_set = TimeseriesGenerator(
  data=test_data,
  targets=test_data,
  length=win_size,
  sampling_rate=sample_step,
  stride=win_moving,
  batch_size=data_batch)

# Model Building

In [17]:
# Create Model
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

# TO-DO: Add more RNN layers & do the validation set
model = Sequential()
model.add(layers.SimpleRNN(10, input_shape=(win_size, dataset.shape[1])))
model.add(layers.Dense(dataset.shape[1], activation="softmax"))
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["acc"])

# Model Training

In [18]:
# Train Model
# Run multiple times of RNN can raise up the accuracy.

# TO-DO: Find the best epochs
model.fit(train_set, epochs=50)

Epoch 1/50
3/3 [==============================] - 1s 8ms/step - loss: 1.3177 - acc: 0.3810
Epoch 2/50
3/3 [==============================] - 0s 5ms/step - loss: 1.2917 - acc: 0.3810
Epoch 3/50
3/3 [==============================] - 0s 7ms/step - loss: 1.2758 - acc: 0.3810
Epoch 4/50
3/3 [==============================] - 0s 6ms/step - loss: 1.2606 - acc: 0.3810
Epoch 5/50
3/3 [==============================] - 0s 6ms/step - loss: 1.2481 - acc: 0.3810
Epoch 6/50
3/3 [==============================] - 0s 6ms/step - loss: 1.2358 - acc: 0.3810
Epoch 7/50
3/3 [==============================] - 0s 6ms/step - loss: 1.2244 - acc: 0.3810
Epoch 8/50
3/3 [==============================] - 0s 6ms/step - loss: 1.2128 - acc: 0.3810
Epoch 9/50
3/3 [==============================] - 0s 5ms/step - loss: 1.2019 - acc: 0.3810
Epoch 10/50
3/3 [==============================] - 0s 7ms/step - loss: 1.1916 - acc: 0.3810
Epoch 11/50
3/3 [==============================] - 0s 6ms/step - loss: 1.1806 - acc: 0.38

# Evaluation

In [20]:
# Model Evaluation
test_loss, test_acc = model.evaluate(test_set)
print("Loss of Test Set:", test_loss)
print("Accuracy of Test Set:", test_acc)

1/1 [==============================] - 0s 282ms/step - loss: 0.8372 - acc: 0.6667
Loss of Test Set: 0.8372073769569397
Accuracy of Test Set: 0.6666666865348816


# Inference

In [21]:
# Model Prediction
Y_pred = np.argmax(model.predict(test_set), axis=-1)
Y_real = np.argmax(test_set[0][1], axis=-1)

# Convert from number to label
print("Prediction:", [mapper[i] for i in Y_pred])
print("Real Value:", [mapper[i] for i in Y_real])

1/1 [==============================] - 0s 34ms/step
Prediction: ['黃金炒飯', '大魯麵', '自助餐', '黃金炒飯', '自助餐', '煎餃']
Real Value: ['自助餐', '大魯麵', '自助餐', '黃金炒飯', '大魯麵', '煎餃']


In [23]:
# User input
from tensorflow.keras.utils import to_categorical

meals = []
meals += [mapper.index(input("請問你前天吃什麼："))]
meals += [mapper.index(input("請問你昨天吃什麼："))]

meals = np.reshape(to_categorical(meals, dataset.shape[1]), (1, win_size, dataset.shape[1]))
meal_today = np.argmax(model.predict(meals), axis=-1)
print("我猜你今天會吃：", mapper[meal_today[0]])

請問你前天吃什麼：黃金炒飯
請問你昨天吃什麼：大魯麵
1/1 [==============================] - 0s 156ms/step
我猜你今天會吃： 煎餃
